In [6]:
import pandas as pd

file_path = r'E:\Downloads\batsmen_rankings.csv'

# Try reading the CSV file
try:
    df = pd.read_csv(file_path, header=None)
    print("File loaded successfully!")
    print("\nCurrent CSV content:")
    print(df)
    print("\nFirst few rows:")
    print(df.head())
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    print("Please check the path and file name.")
except Exception as e:
    print(f"Error occurred: {e}")

File loaded successfully!

Current CSV content:
     0                  1              2
0    1    ,Daryl Mitchell   ,New Zealand
1    2       ,Virat Kohli         ,India
2    3    ,Ibrahim Zadran   ,Afghanistan
3    4      ,Rohit Sharma         ,India
4    5      ,Shubman Gill         ,India
5    6        ,Babar Azam      ,Pakistan
6    7      ,Harry Tector       ,Ireland
7    8         ,Shai Hope   ,West Indies
8    9  ,Charith Asalanka     ,Sri Lanka
9   10          ,KL Rahul         ,India
10  11      ,Shreyas Iyer         ,India
11  12       ,Travis Head     ,Australia
12  13   ,Quinton de Kock  ,South Africa
13  14   ,Pathum Nissanka     ,Sri Lanka
14  15      ,Kusal Mendis     ,Sri Lanka

First few rows:
   0                1             2
0  1  ,Daryl Mitchell  ,New Zealand
1  2     ,Virat Kohli        ,India
2  3  ,Ibrahim Zadran  ,Afghanistan
3  4    ,Rohit Sharma        ,India
4  5    ,Shubman Gill        ,India


In [8]:
print("Shape of the DataFrame (rows, columns):")
print(df.shape)

print("\n" + "="*50 + "\n")

print("Number of columns in DataFrame:")
print(len(df.columns))

print("\n" + "="*50 + "\n")

print("First few rows as raw data:")
for i in range(min(3, len(df))):
    print(f"Row {i}:")
    for col in range(len(df.columns)):
        print(f"  Column {col}: {repr(df.iloc[i, col])}")

Shape of the DataFrame (rows, columns):
(15, 3)


Number of columns in DataFrame:
3


First few rows as raw data:
Row 0:
  Column 0: 1
  Column 1: ',Daryl Mitchell'
  Column 2: ',New Zealand'
Row 1:
  Column 0: 2
  Column 1: ',Virat Kohli'
  Column 2: ',India'
Row 2:
  Column 0: 3
  Column 1: ',Ibrahim Zadran'
  Column 2: ',Afghanistan'


In [9]:
# Create a copy to work with
df_clean = df.copy()

df_clean[1] = df_clean[1].str.lstrip(',').str.strip()

df_clean[2] = df_clean[2].str.lstrip(',').str.strip()

# Assign proper column names
df_clean.columns = ['Rank', 'Name', 'Country']

print("Cleaned DataFrame:")
print(df_clean.head())
print("\n" + "="*50 + "\n")
print("Data types:")
print(df_clean.dtypes)

Cleaned DataFrame:
   Rank            Name      Country
0     1  Daryl Mitchell  New Zealand
1     2     Virat Kohli        India
2     3  Ibrahim Zadran  Afghanistan
3     4    Rohit Sharma        India
4     5    Shubman Gill        India


Data types:
Rank        int64
Name       object
Country    object
dtype: object


In [10]:
# Check for tab characters in the entire DataFrame
has_tabs = df_clean.applymap(lambda x: isinstance(x, str) and '\t' in x).any().any()

if has_tabs:
    print("Tab characters found in the DataFrame. Removing them...")
    # Remove tabs from all string columns
    df_clean = df_clean.applymap(lambda x: x.replace('\t', '') if isinstance(x, str) else x)
else:
    print("No tab characters found in the DataFrame.")

print("\n" + "="*50 + "\n")
print("Final cleaned DataFrame preview:")
print(df_clean.head())

No tab characters found in the DataFrame.


Final cleaned DataFrame preview:
   Rank            Name      Country
0     1  Daryl Mitchell  New Zealand
1     2     Virat Kohli        India
2     3  Ibrahim Zadran  Afghanistan
3     4    Rohit Sharma        India
4     5    Shubman Gill        India


In [11]:
# Define the path for the cleaned file
cleaned_file_path = r'E:\Downloads\batsmen_rankings_cleaned.csv'


df_clean.to_csv(cleaned_file_path, index=False) # Save to CSV

print(f"Cleaned data saved to: {cleaned_file_path}")
print("\n" + "="*50 + "\n")


df_verify = pd.read_csv(cleaned_file_path) # Verify the saved file by reading it back
print("Verification - First 5 rows of saved file:")
print(df_verify.head())

Cleaned data saved to: E:\Downloads\batsmen_rankings_cleaned.csv


Verification - First 5 rows of saved file:
   Rank            Name      Country
0     1  Daryl Mitchell  New Zealand
1     2     Virat Kohli        India
2     3  Ibrahim Zadran  Afghanistan
3     4    Rohit Sharma        India
4     5    Shubman Gill        India


In [13]:
jsonl_file_path = r'E:\Downloads\batsmen_rankings_cleaned.jsonl'

with open(jsonl_file_path, 'w') as f:
    for _, row in df_clean.iterrows():
       
        f.write(row.to_json() + '\n')

print(f"JSONL (newline-delimited JSON) saved to: {jsonl_file_path}")
print("\n" + "="*50 + "\n")

print("First 3 lines of JSONL file:") # Displaying first 3 lines to verify format
with open(jsonl_file_path, 'r') as f:
    for i, line in enumerate(f.readlines()[:3]):
        print(f"Line {i+1}: {line.strip()}")

JSONL (newline-delimited JSON) saved to: E:\Downloads\batsmen_rankings_cleaned.jsonl


First 3 lines of JSONL file:
Line 1: {"Rank":1,"Name":"Daryl Mitchell","Country":"New Zealand"}
Line 2: {"Rank":2,"Name":"Virat Kohli","Country":"India"}
Line 3: {"Rank":3,"Name":"Ibrahim Zadran","Country":"Afghanistan"}


In [14]:
udf_content = """
/**
 * UDF function to transform data for BigQuery.
 * Each line is a JSON object from the JSONL file.
 */
function transform(line) {
    try {
        // Parse the JSON line
        var obj = JSON.parse(line);
        
        // You can transform or validate the data here
        // Example transformations:
        
        // 1. Convert Rank to integer (if not already)
        if (obj.Rank) {
            obj.Rank = parseInt(obj.Rank);
        }
        
        // 2. Ensure Name and Country are strings and trim whitespace
        if (obj.Name) {
            obj.Name = obj.Name.toString().trim();
        }
        
        if (obj.Country) {
            obj.Country = obj.Country.toString().trim();
        }
        
        // 3. Add a processing timestamp (optional)
        obj.processed_timestamp = new Date().toISOString();
        
        // Return as JSON string
        return JSON.stringify(obj);
        
    } catch (e) {
        // Log error and return null to skip this record
        console.error("Error processing line:", e);
        return null;
    }
}
"""

with open('udf.js', 'w') as f:
    f.write(udf_content)

print("UDF file created: udf.js")
print("\n" + "="*50 + "\n")
print("UDF content preview:")
print(udf_content[:500] + "...")

UDF file created: udf.js


UDF content preview:

/**
 * UDF function to transform data for BigQuery.
 * Each line is a JSON object from the JSONL file.
 */
function transform(line) {
    try {
        // Parse the JSON line
        var obj = JSON.parse(line);
        
        // You can transform or validate the data here
        // Example transformations:
        
        // 1. Convert Rank to integer (if not already)
        if (obj.Rank) {
            obj.Rank = parseInt(obj.Rank);
        }
        
        // 2. Ensure Name and Country ...


In [15]:
import json


try:
    with open('bq.json', 'r') as f:
        content = f.read()
        print("Current bq.json content:")
        print(repr(content))  # Show raw content with special characters
        print("\nLength:", len(content))
        
        try:
            parsed = json.loads(content)
            print("\n✅ Valid JSON structure")
            print(json.dumps(parsed, indent=2))
        except json.JSONDecodeError as e:
            print(f"\n❌ Invalid JSON: {e}")
            
except FileNotFoundError:
    print("bq.json file not found locally")

bq.json file not found locally


In [16]:
import json

# Create the correct schema format based on the template example
schema = {
    "BigQuery Schema": [
        {
            "name": "Rank", 
            "type": "INTEGER"
        },
        {
            "name": "Name", 
            "type": "STRING"
        },
        {
            "name": "Country", 
            "type": "STRING"
        },
        {
            "name": "processed_timestamp",
            "type": "TIMESTAMP"
        }
    ]
}

with open('bq.json', 'w') as f:
    json.dump(schema, f, indent=2)

print("✅ Schema file created: bq.json")
print("\n" + "="*60 + "\n")

with open('bq.json', 'r') as f:
    content = f.read()
    print("File content (first 200 chars):")
    print(repr(content[:200]))
    print("\nFull content:")
    print(content)

try:
    with open('bq.json', 'r') as f:
        parsed = json.load(f)
    print("\n✅ Valid JSON structure")
except json.JSONDecodeError as e:
    print(f"\n❌ Invalid JSON: {e}")

✅ Schema file created: bq.json


File content (first 200 chars):
'{\n  "BigQuery Schema": [\n    {\n      "name": "Rank",\n      "type": "INTEGER"\n    },\n    {\n      "name": "Name",\n      "type": "STRING"\n    },\n    {\n      "name": "Country",\n      "type": "STRING"\n    '

Full content:
{
  "BigQuery Schema": [
    {
      "name": "Rank",
      "type": "INTEGER"
    },
    {
      "name": "Name",
      "type": "STRING"
    },
    {
      "name": "Country",
      "type": "STRING"
    },
    {
      "name": "processed_timestamp",
      "type": "TIMESTAMP"
    }
  ]
}

✅ Valid JSON structure


In [17]:
import os

current_directory = os.getcwd()

bq_json_path = os.path.join(current_directory, 'bq.json')

print("📁 Current Working Directory:")
print(f"   {current_directory}")
print("\n" + "="*60 + "\n")

print("📍 Full Path of bq.json:")
print(f"   {bq_json_path}")
print("\n" + "="*60 + "\n")

# Check all files in current directory
print("📂 Files in current directory:")
files = os.listdir(current_directory)
for file in files:
    if file.endswith(('.json', '.jsonl', '.js', '.csv')):
        full_path = os.path.join(current_directory, file)
        size = os.path.getsize(full_path)
        print(f"   ✅ {file} - {size} bytes - Location: {full_path}")

print("\n" + "="*60 + "\n")
print("💡 To upload to Google Cloud Storage, use:")
print("   gsutil cp bq.json gs://bkt-dataflo-metadata/")
print("\n   Or using Python code:")
print("   from google.cloud import storage")
print("   client = storage.Client()")
print("   bucket = client.get_bucket('bkt-dataflo-metadata')")
print("   blob = bucket.blob('bq.json')")
print("   blob.upload_from_filename('bq.json')")

📁 Current Working Directory:
   C:\Users\HP1\Desktop\sem2\Class Test II


📍 Full Path of bq.json:
   C:\Users\HP1\Desktop\sem2\Class Test II\bq.json


📂 Files in current directory:
   ✅ bq.json - 302 bytes - Location: C:\Users\HP1\Desktop\sem2\Class Test II\bq.json
   ✅ Diabetes.csv - 18966 bytes - Location: C:\Users\HP1\Desktop\sem2\Class Test II\Diabetes.csv
   ✅ illinois_vote_2016.csv - 9578 bytes - Location: C:\Users\HP1\Desktop\sem2\Class Test II\illinois_vote_2016.csv
   ✅ udf.js - 1123 bytes - Location: C:\Users\HP1\Desktop\sem2\Class Test II\udf.js


💡 To upload to Google Cloud Storage, use:
   gsutil cp bq.json gs://bkt-dataflo-metadata/

   Or using Python code:
   from google.cloud import storage
   client = storage.Client()
   bucket = client.get_bucket('bkt-dataflo-metadata')
   blob = bucket.blob('bq.json')
   blob.upload_from_filename('bq.json')


In [18]:
import json
import os

target_dir = r"E:\Downloads"

# Create the schema
schema = {
    "BigQuery Schema": [
        {
            "name": "Rank", 
            "type": "INTEGER"
        },
        {
            "name": "Name", 
            "type": "STRING"
        },
        {
            "name": "Country", 
            "type": "STRING"
        },
        {
            "name": "processed_timestamp",
            "type": "TIMESTAMP"
        }
    ]
}

bq_json_path = os.path.join(target_dir, "bq.json")

with open(bq_json_path, 'w') as f:
    json.dump(schema, f, indent=2)

print(f"✅ Schema file created at: {bq_json_path}")
print("\n" + "="*60 + "\n")

if os.path.exists(bq_json_path):
    file_size = os.path.getsize(bq_json_path)
    print(f"File exists: {bq_json_path}")
    print(f"File size: {file_size} bytes")
    
    with open(bq_json_path, 'r') as f:
        content = f.read()
        print("\nFile content preview:")
        print(content[:300] + "..." if len(content) > 300 else content)
else:
    print("❌ File was not created!")

✅ Schema file created at: E:\Downloads\bq.json


File exists: E:\Downloads\bq.json
File size: 302 bytes

File content preview:
{
  "BigQuery Schema": [
    {
      "name": "Rank",
      "type": "INTEGER"
    },
    {
      "name": "Name",
      "type": "STRING"
    },
    {
      "name": "Country",
      "type": "STRING"
    },
    {
      "name": "processed_timestamp",
      "type": "TIMESTAMP"
    }
  ]
}
